# Implementation of the Lasso With High Missing Rate.

The goal of this notebook is to implement the lasso with high missing rate described [here](https://www.ijcai.org/proceedings/2019/0491.pdf). 

## Imports

In [16]:
# Imports
import numpy as np
import pandas as pd

import cvxpy as cp
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

In [17]:
help(Lasso)

Help on class Lasso in module sklearn.linear_model._coordinate_descent:

class Lasso(ElasticNet)
 |  Lasso(alpha=1.0, *, fit_intercept=True, precompute=False, copy_X=True, max_iter=1000, tol=0.0001, warm_start=False, positive=False, random_state=None, selection='cyclic')
 |  
 |  Linear Model trained with L1 prior as regularizer (aka the Lasso).
 |  
 |  The optimization objective for Lasso is::
 |  
 |      (1 / (2 * n_samples)) * ||y - Xw||^2_2 + alpha * ||w||_1
 |  
 |  Technically the Lasso model is optimizing the same objective function as
 |  the Elastic Net with ``l1_ratio=1.0`` (no L2 penalty).
 |  
 |  Read more in the :ref:`User Guide <lasso>`.
 |  
 |  Parameters
 |  ----------
 |  alpha : float, default=1.0
 |      Constant that multiplies the L1 term, controlling regularization
 |      strength. `alpha` must be a non-negative float i.e. in `[0, inf)`.
 |  
 |      When `alpha = 0`, the objective is equivalent to ordinary least
 |      squares, solved by the :class:`LinearR

## HMLasso

In [2]:
class HMLasso():
  """
  Lasso regularization that performs well with high missing rate.

  ------------
  Common uses: Once fitted, the HMLasso can provide linear predictions. 
  It can also be used to select variables of interest from the given data. This 
  second goal can be achieved through selection of variables whose coefficient
  is almost (or equal to) zero.

  ------------
  Parameters:
      mu : float/int, default=1.0: the hyperparameter that control how
      parcimonious the model shall be. The larger mu is, the greater the
      regularization will be (hence the calculated beta_opt might 
      present more nullified coefficients). mu must be positive.

      verbose : float/int, default=1: control how much verbose
      is displayed. Encoded values are 0, 1 and 2. If verbose > 2, there
      will be no difference with verbose=2 display.
  
  ------------
  Methods:
      fit(X, y):
        Fit the HMLasso on (X, y)
        X, the features, must be a mean-centered numpy array of shape (n, p)
        y, the labels, must be a vector of shape (n, 1) or (n,)

        Do not return anything. However, once the fitting is done, one can
        use 'predict' method to predict any given output using the linear model.
      
      predict(X):
        Predict using linear model.
        Return the predicted vector.
  
  ------------
  Constants:
      beta_opt: the estimator.


  """

  def __init__(self, mu=1, verbose=1):

    assert type(mu) is int or type(mu) is float, "mu must be a number."
    assert type(verbose) is int, "verbose must be an integer."
    assert mu >= 0, "mu must be a positive number."

    self.mu = mu
    self.verbose = verbose
    
    self.n = None
    self.p = None
    self.S_pair = None
    self.rho_pair = None
    self.R = None
    self.Sigma_opt = None
    self.beta_opt = None

    self.isFirstProblemSolved = False
    self.isSecondProblemSolved = False
    self.isFitted = False
  
  def predict(self, X):
    """
    Predict using the linear model.

    ------------
    Parameters:
        X : 2D numpy array

    Returns:
        y : 1D numpy array
    """

    assert self.isFitted, "The model has not yet been fitted."
    assert X.shape == (self.n, self.p), f"Given data is of shape {X.shape}. Must be of shape {(self.n, self.p)}"

    return np.dot(X, self.beta_opt)
  
  def fit(self, X, y):
    """
    Fit the HMLasso on (X, y).

    ------------
    Parameters:
        X : 2D numpy array, shape (n,p). It corresponds to the features, and
        must be mean-centered.
        y : 1D numpy array, shape (n,1) or (n,). It corresponds to the lables.

    Returns:
        None
    """
    
    assert type(X) == np.ndarray, "Features are not a numpy array."
    assert type(y) == np.ndarray, "Labels are not a numpy array"
    assert X.shape[0] == y.shape[0], "Features and labels shapes are not compatibles."
    assert len(y.shape) == 1, "Labels are not a vector."

    self.n, self.p = X.shape    
    self.__verify_centering__(X)
    self.S_pair, self.rho_pair, self.R = self.__impute_params__(X, y)
    self.Sigma_opt = self.__solve_first_problem__()

    # It appears that, due to floating points exceptions, Sigma_opt is not always
    # Positive semidefinite. Hence, we shall check it.
    eigenvalues = np.linalg.eig(self.Sigma_opt)[0]
    min_eigenvalue = min(eigenvalues)
    if min_eigenvalue < 0:
      print(f"[Warning] Sigma_opt is not PSD, its minimum eigenvalue is {min_eigenvalue}. Error handled by adding {-min_eigenvalue} to each eigenvalue.")
      self.Sigma_opt = self.Sigma_opt - min_eigenvalue* np.eye(self.p, self.p)
    
    self.beta_opt = self.__solve_second_problem__()

    self.isFitted = True

    if self.verbose > 0:
      print("Model fitted")

  def __verify_centering__(self, X, tolerance=1e-8):
    for col in range(X.shape[1]):
      current_mean = X[:, col].mean()
      if abs(current_mean) > tolerance:
        raise Exception(f"Data is not centered: column {col} has mean of {current_mean}")
  
  def __impute_params__(self, X, y):

    if self.verbose > 0:
      print("[Imputing parameters] Starting...")

    Z = np.nan_to_num(X)
    Y = (Z != 0).astype(int)
    R = np.dot(Y.T, Y)
    if self.verbose > 1:
      print("[Imputing parameters] R calculated.")

    rho_pair = np.divide(np.dot(Z.T, y), R.diagonal())
    if self.verbose > 1:
      print("[Imputing parameters] rho_pair calculated.")

    S_pair = np.divide(np.dot(Z.T, Z), R)
    
    if self.verbose > 1:
      print("[Imputing parameters] S_pair calculated.")

    R = R / self.n

    if self.verbose > 0:
      print("[Imputing parameters] Successfully ended.")

    return S_pair, rho_pair, R


  def __solve_first_problem__(self):
    
    assert self.S_pair is not None, "Pairwise covariance matrix of features is not determined."
    assert self.rho_pair is not None, "Pairwise covariance vector of features and labels is not determined."
    assert self.R is not None, "Weights are not determined."

    Sigma = cp.Variable((self.p, self.p), PSD = True) # Variable to optimize
    obj = cp.Minimize(cp.sum_squares(cp.multiply(self.R, Sigma-self.S_pair))) # Objective to minimize
    constraints = [Sigma >> 0] # Constraints: We want Sigma to be semi definite positive.
    if self.verbose > 1:
      print("[First Problem] Objective and constraints well-defined.")

    # Solve the optimization problem
    prob = cp.Problem(obj, constraints)
    prob.solve()
    if self.verbose > 1:
      print(f"[First Problem] Problem status: {prob.status}.")
    if self.verbose > 0:
      print("[First Problem] Problem solved.")

    self.isFirstProblemSolved = True

    return Sigma.value

  def __solve_second_problem__(self):
    
    assert self.S_pair is not None, "Pairwise covariance matrix of features is not determined."
    assert self.rho_pair is not None, "Pairwise covariance vector of features and labels is not determined."
    assert self.R is not None, "Weights are not determined."
    assert self.isFirstProblemSolved, " First optimization problem has not been solved."
    assert self.Sigma_opt is not None, "Sigma_opt is unknown. First optimization problem might have not been solved."
    
    beta = cp.Variable(self.p) # Variable to optimize
    obj = cp.Minimize(0.5 * cp.quad_form(beta, self.Sigma_opt) - self.rho_pair.T @ beta + self.mu * cp.norm1(beta)) # Objective to minimize
    constraints = [] # Constraints
    if self.verbose > 1:
      print("[Second Problem] Objective and constraints well-defined.")

    # Solve the optimization problem
    prob = cp.Problem(obj, constraints)
    prob.solve()
    if self.verbose > 1:
      print(f"[Second Problem] Problem status: {prob.status}.")
    if self.verbose > 0:
      print("[Second Problem] Problem solved.\n")
    
    self.isFirstProblemSolved = True

    return beta.value

In [3]:
# Test

def get_Xy(n, p, replace_rate=0.3):
  X = 100*np.random.rand(n,p) # Generate random X
  y = 7*X[:, 0] - 2 * X[:, 1] + 5 * X[:, 2] + 19 * X[:, 3] + 6*X[:, 4]
  
  indices = np.full(X.shape, False, bool)
  mask = np.random.choice([False, True], size=X.shape, p=((1 - replace_rate), replace_rate))
  X[mask] = np.nan

  return X, y

X, y = get_Xy(15000, 1500, 0.4)

scaler = StandardScaler(with_std=False)
X_scaled = scaler.fit_transform(X)

In [4]:
lasso = HMLasso(mu=1, verbose=2)
lasso.fit(X_scaled, y)

[Imputing parameters] Starting...
[Imputing parameters] Z calculated.
[Imputing parameters] Y calculated.
[Imputing parameters] R calculated.
[Imputing parameters] rho_pair calculated.
[Imputing parameters] S_pair calculated.
[Imputing parameters] Successfully ended.
[First Problem] Objective and constraints well-defined.
[First Problem] Problem status: optimal.
[First Problem] Problem solved.
[Second Problem] Objective and constraints well-defined.
[Second Problem] Problem status: optimal.
[Second Problem] Problem solved.

Model fitted
